In [11]:
import pandas as pd
from collections import Counter

class HandOfCards():
    card_order_without_jokers = {'A':14, 'K':13, 'Q':12, 'J':11, 'T':10, '9':9, '8':8, '7':7, '6':6, '5':5, '4':4, '3':3, '2':2}
    card_order_with_jokers = {'A':14, 'K':13, 'Q':12, 'T':10, '9':9, '8':8, '7':7, '6':6, '5':5, '4':4, '3':3, '2':2, 'J':1}

    def __init__(self, hand, bid, jokers=False):
        self.hand = list(hand)
        self.bid = bid
        self.jokers = jokers
        self.type = self.hand_type()
        self.card_order = self.card_order_without_jokers if not jokers else self.card_order_with_jokers

    def hand_type(self):

        counts = Counter(self.hand)
        counts_values = list(counts.values())
        counts_values.sort(reverse=True)

        if counts_values == [5]:
            return 7
        elif counts_values == [4, 1]:
            if self.jokers and (counts['J'] == 1 or counts['J'] == 4):
                return 7
            return 6
        elif counts_values == [3, 2]:
            if self.jokers and (counts['J'] == 2 or counts['J'] == 3):
                return 7
            return 5
        elif counts_values == [3, 1, 1]:
            if self.jokers and (counts['J'] == 1 or counts['J'] == 3):
                return 6
            return 4
        elif counts_values == [2, 2, 1]:
            if self.jokers and counts['J'] == 1:
                return 5
            if self.jokers and counts['J'] == 2:
                return 6
            return 3
        elif counts_values == [2, 1, 1, 1]:
            if self.jokers and (counts['J'] == 1 or counts['J'] == 2):
                return 4
            return 2
        else:
            if self.jokers and counts['J'] == 1:
                return 2
            return 1


    def __lt__(self, other):
        if self.type == other.type:
            for i, card in enumerate(self.hand):
                if self.card_order[card] != self.card_order[other.hand[i]]:
                    return self.card_order[card] < self.card_order[other.hand[i]]
        return self.type < other.type

    def __gt__(self, other):
        if self.type == other.type:
            for i, card in enumerate(self.hand):
                if self.card_order[card] != self.card_order[other.hand[i]]:
                    return self.card_order[card] > self.card_order[other.hand[i]]
        return self.type > other.type

    def __eq__(self, other):
        if self.type == other.type:
            for i, card in enumerate(self.hand):
                if self.card_order[card] != self.card_order[other.hand[i]]:
                    return False
            return True

    def __ne__(self, other):
        return self.type != other.type
    
    def __str__(self):
        return "hand: " + str(self.hand) + " bid: " + str(self.bid) + " value: " + str(self.value)

In [12]:
# load data
df = pd.read_csv('data/day07_input.txt', sep=" ", header=None)
df.columns = ['hand', 'bid']

# create list of hands of cards
handsOfCards = list([handOfCard for handOfCard in df.apply(lambda row: HandOfCards(row['hand'], row['bid']), axis=1)])
handsOfCards.sort(reverse=True)

sum = 0
rank = len(handsOfCards)

for handOfCard in handsOfCards:
    # print(handOfCard)
    sum += handOfCard.bid * rank
    rank -= 1

print(sum)

249204891


## Part two

To make things a little more interesting, the Elf introduces one additional rule. Now, `J` cards are jokers - wildcards that can act like whatever card would make the hand the strongest type possible.

To balance this, `J` cards are now the weakest individual cards, weaker even than `2`. The other cards stay in the same order: `A`, `K`, `Q`, `T`, `9`, `8`, `7`, `6`, `5`, `4`, `3`, `2`, `J`.

J cards can pretend to be whatever card is best for the purpose of determining hand type; for example, QJJQ2 is now considered four of a kind. However, for the purpose of breaking ties between two hands of the same type, J is always treated as `J`, not the card it's pretending to be: `JKKK2` is weaker than `QQQQ2` because `J` is weaker than `Q`.

Now, the above example goes very differently:

```
32T3K 765
T55J5 684
KK677 28
KTJJT 220
QQQJA 483
```

- `32T3K` is still the only one pair; it doesn't contain any jokers, so its strength doesn't increase.
- `KK677` is now the only two pair, making it the second-weakest hand.
- `T55J5`, `KTJJT`, and `QQQJA` are now all four of a kind! `T55J5` gets rank 3, `QQQJA` gets rank 4, and `KTJJT` gets rank 5.

With the new joker rule, the total winnings in this example are `5905`.

Using the new joker rule, find the rank of every hand in your set. What are the new total winnings?

In [13]:
# load data
df = pd.read_csv('data/day07_input.txt', sep=" ", header=None)
df.columns = ['hand', 'bid']

# create list of hands of cards
handsOfCards = list([handOfCard for handOfCard in df.apply(lambda row: HandOfCards(row['hand'], row['bid'], jokers=True), axis=1)])
handsOfCards.sort(reverse=True)

sum = 0
rank = len(handsOfCards)

for handOfCard in handsOfCards:
    # print(handOfCard)
    sum += handOfCard.bid * rank
    rank -= 1

print(sum)

249666369
